In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Aug 15 19:16:39 2017

@author: lile
"""

'\nCreated on Tue Aug 15 19:16:39 2017\n\n@author: lile\n'

# 知识点
"""
1. groupby
2. map
3. lambda
4. defaultdict(factory_type)
5. str.format() # 格式化字符串
6. tf.where()
7. tf.map_fn()

"""

In [2]:
import tensorflow as tf
import glob

In [3]:
sess = tf.InteractiveSession()

# 创建 .TFRecords 文件

In [4]:
image_filenames = glob.glob("home/lile/imagenet-dogs/n02*/*.jpg")

from itertools import groupby
from collections import defaultdict

training_dataset = defaultdict(list)   # 
testing_dataset  = defaultdict(list)

# breed:属; 种类; 类型;
# 返回[类型，文件路径]对的列表
image_filename_with_breed = list(map(lambda filename:(filename.split("/")[2],filename),
                                                 image_filenames))

#将key函数作用于原循环器的各个元素,根据key函数结果,
#将拥有相同函数结果的元素分到一个新的循环器。每个新的循环器以函数返回结果为标签

for dog_breed, breed_images in groupby(image_filename_with_breed ,lambda x:x[0]):
    for i, breed_image in enumerate(breed_images):
        if i % 5 == 0:
            testing_dataset[dog_breed].append(breed_image[1])
        else:
            training_dataset[dog_breed].append(breed_image[1])
            
#检查每个品种的测试图像是否至少有全部图像的18%
    breed_training_count = len(training_dataset[dog_breed])
    breed_testing_count = len(testing_dataset[dog_breed])

    assert round(breed_testing_count / (breed_training_count + breed_testing_count), 2) > 0.18, "Not enough testing images"

def write_records_file(dataset, record_location):
    """
    用  dataset 中的图像填充一个TFRecord文件，并将其类别包含进来

    Parameters
    ----------
    dataset : dict(list)
      Dictionary with each key being a label for the list of image filenames of its value.
    record_location : str
      Location to store the TFRecord output.
    """
    writer = None

    # 枚举dataset， 每个TFRecord文件记录100副图像，以加快写操作
    current_index = 0
    for breed, images_filenames in dataset.items():
        for image_filename in images_filenames:
            if current_index % 100 == 0:
                if writer:
                    writer.close()
                
                #格式化字符串
                record_filename = "{record_location}-{current_index}.tfrecords".format(
                    record_location=record_location,
                    current_index=current_index)

                writer = tf.python_io.TFRecordWriter(record_filename)
            current_index += 1

            image_file = tf.read_file(image_filename)

            # In ImageNet dogs, there are a few images which TensorFlow doesn't recognize as JPEGs. This
            # try/catch will ignore those images.
            # 忽略掉tensorflow不能识别的jpeg图像，使用try/catch语句
            try:
                image = tf.image.decode_jpeg(image_file)
            except:
                print(image_filename)
                continue

            # Converting to grayscale saves processing and memory but isn't required.
            # 灰度变换（不是必须的）
            grayscale_image = tf.image.rgb_to_grayscale(image)
            resized_image = tf.image.resize_images(grayscale_image, 250, 151)

            # 在这里使用tf.cast,是因为虽然尺寸更改后的图像的数据类型三浮点型，但RGB值尚未转换到[0,1]区间内??
            # tf.cast 并不进行缩放
            image_bytes = sess.run(tf.cast(resized_image, tf.uint8)).tobytes()

            # Instead of using the label as a string, it'd be more efficient to turn it into either an
            # integer index or a one-hot encoded rank one tensor.
            # https://en.wikipedia.org/wiki/One-hot
            # 推荐将label转换为整型或one-hot编码，这将更高效，此处还是将label按字符串使用
            image_label = breed.encode("utf-8")

            example = tf.train.Example(features=tf.train.Features(feature={
                'label': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_label])),
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes]))
            }))

            writer.write(example.SerializeToString())
    writer.close()

In [5]:
# 创建TFRecord 文件
write_records_file(testing_dataset, "home/lile/output/testing-images/testing-image")
write_records_file(training_dataset, "home/lile/output/training-images/training-image")

AttributeError: 'NoneType' object has no attribute 'close'

# 载入图像

In [ ]:
filename_queue = tf.train.string_input_producer(
    tf.train.match_filenames_once("home/lile/output/training-images/*.tfrecords"))
reader = tf.TFRecordReader()
# serialized 指向该文件中的下一个图像
_, serialized = reader.read(filename_queue)

features = tf.parse_single_example(
    serialized,
    features={
        'label': tf.FixedLenFeature([], tf.string),
        'image': tf.FixedLenFeature([], tf.string),
    })

record_image = tf.decode_raw(features['image'], tf.uint8)

# Changing the image into this shape helps train and visualize the output by converting it to
# be organized like an image.
# 图像已经灰度化了
image = tf.reshape(record_image, [250, 151, 1])

label = tf.cast(features['label'], tf.string)

min_after_dequeue = 10
batch_size = 3
capacity = min_after_dequeue + 3 * batch_size
image_batch, label_batch = tf.train.shuffle_batch(
    [image, label], batch_size=batch_size, num_threads = 4, capacity=capacity, min_after_dequeue=min_after_dequeue)

# 定义模型

In [ ]:
# 将图像转换为float32型,并归一化到[0,1]
float_image_batch = tf.image.convert_image_dtype(image_batch, tf.float32)

## 第一层
    32个5x5大小的卷积核

In [ ]:
conv2d_layer_one = tf.contrib.layers.convolution2d(
    float_image_batch,
    num_output_channels=32,     # 滤波器数量
    kernel_size=(5,5),          # It's only the filter height and width.
    activation_fn=tf.nn.relu,
    weight_init=tf.random_normal,
    stride=(2, 2),
    trainable=True)

pool_layer_one = tf.nn.max_pool(conv2d_layer_one,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='SAME')

# Note, the first and last dimension of the convolution output hasn't changed but the
# middle two dimensions have.
conv2d_layer_one.get_shape(), pool_layer_one.get_shape()

## 第二层
    64个5x5大小卷积核

In [ ]:
conv2d_layer_two = tf.contrib.layers.convolution2d(
    pool_layer_one,
    num_output_channels=64,        # More output channels means an increase in the number of filters
    kernel_size=(5,5),
    activation_fn=tf.nn.relu,
    weight_init=tf.random_normal,
    stride=(1, 1),
    trainable=True)

pool_layer_two = tf.nn.max_pool(conv2d_layer_two,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='SAME')

conv2d_layer_two.get_shape(), pool_layer_two.get_shape()

## 第一全连接层
    512个节点

In [ ]:
flattened_layer_two = tf.reshape(
    pool_layer_two,
    [
        batch_size,  # Each image in the image_batch
        -1           # Every other dimension of the input
    ])

flattened_layer_two.get_shape()

# The weight_init parameter can also accept a callable, a lambda is used here  returning a truncated normal
# with a stddev specified.
hidden_layer_three = tf.contrib.layers.fully_connected(
    flattened_layer_two,
    512,
    weight_init=lambda i, dtype: tf.truncated_normal([38912, 512], stddev=0.1),
    activation_fn=tf.nn.relu
)

# dropout层
    prob = 0.1

In [ ]:
# Dropout some of the neurons, reducing their importance in the model
hidden_layer_three = tf.nn.dropout(hidden_layer_three, 0.1)

## 第二全连接层
    120个节点，即120个类目

In [ ]:
# The output of this are all the connections between the previous layers and the 120 different dog breeds
# available to train on.
final_fully_connected = tf.contrib.layers.fully_connected(
    hidden_layer_three,
    120,  # Number of dog breeds in the ImageNet Dogs dataset
    weight_init=lambda i, dtype: tf.truncated_normal([512, 120], stddev=0.1)
)

# 训练

## 先将label由字符串形式转换为数字形式

In [ ]:
# Find every directory name in the imagenet-dogs directory (n02085620-Chihuahua, ...)
# c.split("/")[-1]: -1表示最后一个‘/’
labels = list(map(lambda c: c.split("/")[-1], glob.glob("home/lile/imagenet-dogs/*")))

# Match every label from label_batch and return the index where they exist in the list of classes
train_labels = tf.map_fn(lambda l: tf.where(tf.equal(labels, l))[0,0:1][0], label_batch, dtype=tf.int64)

In [ ]:
## 定义损失函数

In [ ]:
# setup-only-ignore
loss = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(
        final_fully_connected, train_labels))

batch = tf.Variable(0)
learning_rate = tf.train.exponential_decay(
    0.01,
    batch * 3,
    120,
    0.95,
    staircase=True)

optimizer = tf.train.AdamOptimizer(
    learning_rate, 0.9).minimize(
    loss, global_step=batch)

train_prediction = tf.nn.softmax(final_fully_connected)